# Langchain crash course

In [1]:
import os
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

## LLMs

In [4]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)



"Spice Palace"


In [5]:
llm("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n\n"Raj Mahal Fine Indian Cuisine"'

## Prompt Templates

In [6]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

# print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [7]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

/Users/brianroepke/Projects/deadpool-llm/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n"El Sabroso Cocina Mexicana"'

In [8]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


'\n"La Cantina de México"'

In [9]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [10]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



1. Tandoori Chicken: Marinated chicken cooked in a clay oven with a blend of spices and served with naan bread.

2. Butter Chicken: Tender chicken in a creamy tomato and butter-based sauce, served with basmati rice.

3. Lamb Vindaloo: Spicy and tangy curry with tender pieces of lamb, served with garlic naan.

4. Vegetable Biryani: Fragrant basmati rice cooked with mixed vegetables, herbs, and spices, served with raita.

5. Chana Masala: Chickpeas cooked in a rich tomato and onion gravy, served with fluffy naan bread.

6. Palak Paneer: Creamy spinach and cottage cheese curry, served with basmati rice.

7. Chicken Tikka Masala: Grilled chicken cooked in a creamy tomato and onion gravy, served with garlic naan.

8. Aloo Gobi: Cauliflower and potato curry cooked in a blend of spices, served with basmati rice.

9. Samosas: Crispy pastry filled with spiced potatoes and peas, served with chutney.

10. Mango Lassi: A refreshing yogurt-based drink with mango puree and spices.


#### Sequential Chain

In [11]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [12]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [13]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [15]:
chain.invoke({"cuisine": "Indian"})

{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Maharaja\'s Palace - Indian Fine Dining" ',
 'menu_items': '\n\n1. Tandoori Chicken Tikka: Marinated chicken pieces cooked in a clay oven and served with mint chutney and naan bread.\n\n2. Palak Paneer: Creamy spinach and cottage cheese curry.\n\n3. Butter Chicken: Tender chicken pieces cooked in a rich and creamy tomato sauce.\n\n4. Lamb Rogan Josh: Slow-cooked lamb in a spicy tomato and onion gravy.\n\n5. Vegetable Biryani: Fragrant basmati rice cooked with mixed vegetables, herbs, and spices.\n\n6. Malai Kofta: Vegetable and cheese dumplings in a creamy cashew and tomato gravy.\n\n7. Dal Makhani: Black lentils slow-cooked in a creamy tomato and butter sauce.\n\n8. Garlic Naan: Traditional Indian bread infused with garlic and baked in a clay oven.\n\n9. Samosas: Crispy pastry filled with spiced potatoes and peas, served with tamarind chutney.\n\n10. Gulab Jamun: Sweet and syrupy fried dough balls served with ice cream.\n\n11. Saffron Ri

## Agents

In [14]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key

os.environ['SERPAPI_API_KEY'] = "add your serpapi key here"

#### serpapi and llm-math tool

In [15]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")



> Entering new  chain...
 I need to find the GDP of US in 2022
Action: Search
Action Input: US GDP in 2022
Observation: $25.46 trillion
Thought: I need to add 5 to this number
Action: Calculator
Action Input: 25.46 + 5
Observation: Answer: 30.46
Thought: I now know the final answer
Final Answer: The GDP of US in 2022 plus 5 is $30.46 trillion.

> Finished chain.


'The GDP of US in 2022 plus 5 is $30.46 trillion.'

#### Wikipedia and llm-math tool

In [16]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new  chain...
 I need to find out when Elon Musk was born and then calculate his age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a business magnate and investor. He is the founder, CEO, and chief engineer of SpaceX; angel investor, CEO, and product architect of Tesla, Inc.; owner, CTO, and chairman of Twitter; founder of the Boring Company and X Corp.; co-founder of Neuralink and OpenAI; and president of the philanthropic Musk Foundation. Musk is the wealthiest person in the world according to the Bloomberg Billionaires Index and Forbes's Real Time Billionaires list as of June 2023, primarily from his ownership stakes in Tesla and SpaceX, with an estimated net worth of around $225 billion according to Bloomberg and $230.2 billion according to Forbes.Musk was born in Pretoria, South Africa, and briefly attended the University of Pretoria before moving to Canada at age 18, acquiring citizen

'Elon Musk was born on June 28, 1971 and is currently 52 years old in 2023.'

## Memory

In [17]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [18]:
name = chain.run("Indian")
print(name)



Maharaja's Palace Cuisine


In [19]:
chain.memory

In [20]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [21]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [22]:
name = chain.run("Arabic")
print(name)



Al-Fez Restaurant


In [23]:
print(chain.memory.buffer)

Human: Mexican
AI: 

Taco Fiesta
Human: Arabic
AI: 

Al-Fez Restaurant


#### ConversationChain

In [24]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [25]:
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was won by the West Indies in 1975.'

In [26]:
convo.run("How much is 5+5?")

' 10.'

In [27]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the West Indies team that won the first Cricket World Cup in 1975 was Clive Lloyd.'

In [29]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first Cricket World Cup was won by the West Indies in 1975.
Human: How much is 5+5?
AI:  10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the West Indies team that won the first Cricket World Cup in 1975 was Clive Lloyd.


#### ConversationBufferWindowMemory

In [30]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was held in 1975 and the winner was the West Indies.'

In [31]:
convo.run("How much is 5+5?")

' 5+5 is 10.'

In [32]:
convo.run("Who was the captain of the winning team?")

" I'm sorry, I don't know."